In [6]:
# storing data files in an array
import os
data_folder_name = 'Kickstarter_2021-03-18T03_20_11_507Z'
data_files = []
for root, dirs, files in os.walk(data_folder_name):
    for file in files:
        data_files.append(os.path.join(root, file))

# sorted based on file names
data_files = sorted(data_files)
print(data_files)

['Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter001.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter002.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter003.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter004.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter005.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter006.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter007.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter008.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter009.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter010.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter011.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter012.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter013.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter014.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter015.csv', 'Kickstarter_2021-03-18T03_20_11_507Z/Kickstarter016.csv',

In [18]:
# import panda and numpy
import pandas as pd
import numpy as np


# read the first data file
# df = pd.read_csv(data_files[0])

# read all the files at one go
df = pd.concat(pd.read_csv(f) for f in data_files)

# get the info about the dimensions
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213583 entries, 0 to 3446
Data columns (total 38 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   backers_count             213583 non-null  int64  
 1   blurb                     213576 non-null  object 
 2   category                  213583 non-null  object 
 3   converted_pledged_amount  213583 non-null  int64  
 4   country                   213583 non-null  object 
 5   country_displayable_name  213583 non-null  object 
 6   created_at                213583 non-null  int64  
 7   creator                   213583 non-null  object 
 8   currency                  213583 non-null  object 
 9   currency_symbol           213583 non-null  object 
 10  currency_trailing_code    213583 non-null  bool   
 11  current_currency          213583 non-null  object 
 12  deadline                  213583 non-null  int64  
 13  disable_communication     213583 non-null  boo

In [19]:
df['state'].value_counts()

successful    125856
failed         72744
canceled        8960
live            6023
Name: state, dtype: int64